GODEL - (Grounded Open
Dialogue Language Model https://www.microsoft.com/en-us/research/uploads/prod/2022/05/2206.11309.pdf

# Step 0: Install 🤗 `transformers` & `gradio`

In [1]:
! pip install transformers gradio -q

You should consider upgrading via the '/Users/macbookpro/.pyenv/versions/3.8.12/envs/godel-env/bin/python3.8 -m pip install --upgrade pip' command.


# Step 1 — Setting up the Chatbot Model - Microsoft GODEL

In [11]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/GODEL-v1_1-base-seq2seq")
model = AutoModelForSeq2SeqLM.from_pretrained("microsoft/GODEL-v1_1-base-seq2seq")


# Step 2 — Defining a `predict` function with `state` and model prediction

In [9]:
def predict(knowledge, input, history=[]):

    #instruction = 'Instruction: given a dialog context and related knowledge, you need to response safely  only based on the supplied knowledge.'

    #instruction = 'Instruction: given a dialog context, you need to response empathically.'
    
    instruction = 'Instruction: given a dialog context and related knowledge, you need to answer the question based on the knowledge.'
    

    if knowledge != '':
      knowledge = '[KNOWLEDGE] ' + knowledge

    dialog = ' EOS ' .join([input])


    query = f"{instruction} [CONTEXT] {dialog} {knowledge}"

    top_p = 0.9
    min_length = 8
    max_length = 64


    # tokenize the new input sentence
    user_input_ids = tokenizer.encode(f"{query}", return_tensors='pt')


    output = model.generate(user_input_ids, min_length=int(
        min_length), max_length=int(max_length), top_p=top_p, do_sample=True).tolist()
    
  
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    #print(response)
    history.append((input, response))

    return history, history

# Step 3 — Creating a Gradio Chatbot UI 

In [10]:
import gradio as gr

gr.Interface(fn=predict,
             #inputs=["text",'state'],  
             inputs=[gr.Text(label='knowledge'), "text", 'state'],
             outputs=["chatbot",'state']).launch(debug = True, share = True)


Running on local URL:  http://127.0.0.1:7860
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Running on public URL: https://146dd1808be182a54d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://146dd1808be182a54d.gradio.live
